## Quarterly billing or new project report 
* Pull a quarterly billing or new project report from the TIRCP spreadsheet of any new allocations (new project IDs) made since the previous report?
* New projects highlighted in yellow
* Sort small Project ID to larger project ID
* Include existing projects (no highlight)
* Each cycle its own tab!


In [110]:
import A1_data_prep
import A2_tableau
import numpy as np
import pandas as pd
from babel.numbers import format_currency
from calitp import *

pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [111]:
# Load in sheets
project = A1_data_prep.clean_project()
alloc = A1_data_prep.clean_allocation()

/opt/conda/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [123]:
# Columns for allocation subset
alloc_subset = [
    "allocation_award_year",
    "allocation_ppno",
    "allocation_project_id",
    "allocation_ea",
    "allocation_grant_recipient",
    "allocation_phase",
    "allocation_allocation_amount",
    "allocation_sb1_funding",
    "allocation_sb1_budget_year",
    "allocation_ggrf_funding",
    "allocation_ggrf_budget_year",
    "allocation_allocation_date",
]

In [124]:
# Subset
project2 = project[
    ['project_grant_recipient',"project_project_title", "project_ppno", "project_district", "project_award_year"]
]
alloc2 = alloc[alloc_subset]

In [125]:
# Merge the 2
m1 = pd.merge(
    project2,
    alloc2,
    how="inner",
    left_on=["project_ppno", "project_award_year"],
    right_on=["allocation_ppno", "allocation_award_year"],
)

In [126]:
# Melt based on project title
m1_T = pd.melt(
    m1,
    id_vars=[
        "project_award_year",
        "allocation_project_id",
        "allocation_ea",
        "project_district",
        'project_grant_recipient',
        "project_project_title",
        "allocation_allocation_amount",
        "allocation_allocation_date",
        'allocation_sb1_budget_year',
        'allocation_ggrf_budget_year',
    ],
    value_vars=["allocation_sb1_funding", "allocation_ggrf_funding"],
)

In [127]:
# Keep only values above 1
m1_T = (m1_T.loc[m1_T["value"] > 0.00]).reset_index(drop=True)